In [1]:
import json
import random
import os
import shutil
import socket
import pprint
from tqdm import tqdm

In [2]:
in_nimble = False
in_WSL = False
in_tpu = False
in_pi = False

host = socket.gethostname()
if('cuda' in host):
    in_nimble = True
    print("In NimbleBox")
if(host == 'LTsuphale-NC2JM'):
    in_WSL = True
    print("In WSL")    
if(host == 'raspberrypi'):
    in_pi = True
    print("In raspberry-pi")    

In WSL


In [3]:
copy_required = True
# total_test_images = 118287
total_test_images   = 128

if(in_WSL == True):
    coco_dir = '/home/suphale/snehal_bucket/coco/raw-data'
    annotation_dir = coco_dir + '/annotations'
    train_2017_dir = coco_dir + '/train2017'
    captions_val2017_json = annotation_dir + '/captions_val2017.json'
    captions_train2017_json = annotation_dir + '/captions_train2017.json'
    test_image_dir = '/home/suphale/WorkSpace/test_images'
#     test_image_dir = train_2017_dir
    test_image_captions_txt = test_image_dir + '/captions.txt'
    copy_required = True
if(in_nimble == True):
    coco_dir = '/mnt/disks/user/project/coco'
    annotation_dir = coco_dir + '/annotations'
    train_2017_dir = coco_dir + '/train2017'
    captions_val2017_json = annotation_dir + '/captions_val2017.json'
    captions_train2017_json = annotation_dir + '/captions_train2017.json'
    # test_image_dir = '/mnt/disks/user/project/WorkSpace/test_images'
    test_image_dir = train_2017_dir
    copy_required = False
if(in_nimble == True):
    coco_dir = '/mnt/disks/user/project/coco'
    annotation_dir = coco_dir + '/annotations'
    train_2017_dir = coco_dir + '/train2017'
    captions_val2017_json = annotation_dir + '/captions_val2017.json'
    captions_train2017_json = annotation_dir + '/captions_train2017.json'
    # test_image_dir = '/mnt/disks/user/project/WorkSpace/test_images'
    test_image_dir = train_2017_dir
    copy_required = True

test_image_captions_txt = './lists/captions_' + str(total_test_images) + '.txt'
test_image_list_txt = './lists/images_' + str(total_test_images) + '.txt'
if(test_image_dir == train_2017_dir):
    copy_required = False
print(test_image_captions_txt)

./lists/captions_128.txt


In [4]:
captions_train2017_json = annotation_dir + '/captions_train2017.json'
f = open(captions_train2017_json,)
captions_train2017_data = json.load(f)
f.close()

In [5]:
instances_val2017_json = annotation_dir + '/instances_train2017.json'
f = open(instances_val2017_json,)
instances_val2017_data = json.load(f)
f.close()


In [7]:
for key in captions_train2017_data.keys():
    print("key=%s type=%s" % (key,str(type(captions_train2017_data[key]))))
    if( type(captions_train2017_data[key]) == dict):
        for subkey in captions_train2017_data[key].keys():
            print("    key=%s %s" % (subkey,str(type(captions_train2017_data[key][subkey]))))
    if( type(captions_train2017_data[key]) == list):
        print("    %s" % (str(captions_train2017_data[key][0])))


key=info type=<class 'dict'>
    key=description <class 'str'>
    key=url <class 'str'>
    key=version <class 'str'>
    key=year <class 'int'>
    key=contributor <class 'str'>
    key=date_created <class 'str'>
key=licenses type=<class 'list'>
    {'url': 'http://creativecommons.org/licenses/by-nc-sa/2.0/', 'id': 1, 'name': 'Attribution-NonCommercial-ShareAlike License'}
key=images type=<class 'list'>
    {'license': 3, 'file_name': '000000391895.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000391895.jpg', 'height': 360, 'width': 640, 'date_captured': '2013-11-14 11:18:45', 'flickr_url': 'http://farm9.staticflickr.com/8186/8119368305_4e622c8349_z.jpg', 'id': 391895}
key=annotations type=<class 'list'>
    {'image_id': 203564, 'id': 37, 'caption': 'A bicycle replica with a clock as the front wheel.'}


In [8]:
for key in instances_val2017_data.keys():
    print("key=%s type=%s" % (key,str(type(instances_val2017_data[key]))))
    if( type(instances_val2017_data[key]) == dict):
        for subkey in instances_val2017_data[key].keys():
            print("    key=%s %s" % (subkey,str(type(instances_val2017_data[key][subkey]))))
    if( type(instances_val2017_data[key]) == list):
        print("    %s" % (str(instances_val2017_data[key][0])))


key=info type=<class 'dict'>
    key=description <class 'str'>
    key=url <class 'str'>
    key=version <class 'str'>
    key=year <class 'int'>
    key=contributor <class 'str'>
    key=date_created <class 'str'>
key=licenses type=<class 'list'>
    {'url': 'http://creativecommons.org/licenses/by-nc-sa/2.0/', 'id': 1, 'name': 'Attribution-NonCommercial-ShareAlike License'}
key=images type=<class 'list'>
    {'license': 3, 'file_name': '000000391895.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000391895.jpg', 'height': 360, 'width': 640, 'date_captured': '2013-11-14 11:18:45', 'flickr_url': 'http://farm9.staticflickr.com/8186/8119368305_4e622c8349_z.jpg', 'id': 391895}
key=annotations type=<class 'list'>
    {'segmentation': [[239.97, 260.24, 222.04, 270.49, 199.84, 253.41, 213.5, 227.79, 259.62, 200.46, 274.13, 202.17, 277.55, 210.71, 249.37, 253.41, 237.41, 264.51, 242.54, 261.95, 228.87, 271.34]], 'area': 2765.1486500000005, 'iscrowd': 0, 'image_id': 558840, 'bbox'

In [14]:
id=262508
my_dict = {}
my_dict["captions"] = []
pp = pprint.PrettyPrinter(indent=4)
for x in instances_val2017_data["images"]:
    if(x["id"] == id):
        # pp.pprint(x)
        my_dict["images"] = x

for x in instances_val2017_data["annotations"]:
    if(x["image_id"] == id):
        # pp.pprint(x)
        my_dict["annotations"] = x

for x in captions_train2017_data["annotations"]:
    if(x["image_id"] == id):
        # pp.pprint(x)
        my_dict["captions"].append(x['caption'])

pp.pprint(my_dict)

{   'annotations': {   'area': 1157.0981500000003,
                       'bbox': [240.43, 255.25, 38.58, 50.33],
                       'category_id': 40,
                       'id': 1470861,
                       'image_id': 262508,
                       'iscrowd': 0,
                       'segmentation': [   [   240.43,
                                               255.25,
                                               248.51,
                                               259.65,
                                               255.49,
                                               258.18,
                                               279.01,
                                               290.89,
                                               274.6,
                                               300.81,
                                               267.98,
                                               305.22,
                                               261.37,
            

In [20]:
for x in data["info"].keys():
    print("key=%s %s" % (x,str(type(data["info"][x]))))
    # print(data[x][0])

key=description <class 'str'>
key=url <class 'str'>
key=version <class 'str'>
key=year <class 'int'>
key=contributor <class 'str'>
key=date_created <class 'str'>


In [4]:
f = open(captions_train2017_json,)
data = json.load(f)

In [5]:
if(copy_required == True):
    shutil.rmtree(test_image_dir,ignore_errors = True)
    os.mkdir(test_image_dir)
captions_file = open(test_image_captions_txt, "w")
image_list_file = open(test_image_list_txt, "w")

In [6]:
total_images = len(data['images'])
print("total_images = %d" % (total_images))
print("total_test_images = %d" % (total_test_images))

if (total_images < total_test_images):
    total_test_images = total_images
print("total images = %d total test images = %d" % (total_images,total_test_images))
random_list = random.choices(data['images'],k=total_test_images)
for x in tqdm(random_list):
    file_name= x['file_name']
    coco_url= x['coco_url']
    image_id = x['id']
    # print(file_name)
    # print(coco_url)
    image_list_file.write(file_name+'\n')
    for a in data['annotations']:
        if(a['image_id'] == image_id):
            #print(file_name + ',' + a['caption'])
            # print(train_2017_dir + '/' + file_name)
            if(copy_required == True):
                shutil.copyfile(train_2017_dir + '/' + file_name, test_image_dir + '/' + file_name)
            captions_file.write(file_name + ',' + a['caption']+'\n')

captions_file.close()
image_list_file.close()


  1%|          | 1/128 [00:00<00:15,  7.95it/s]

total_images = 118287
total_test_images = 128
total images = 118287 total test images = 128


100%|██████████| 128/128 [00:10<00:00, 11.73it/s]


In [7]:
# data['images'][0]